In [207]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [208]:
import numpy as np
import pandas as pd

import os
import re
import warnings
print(os.listdir("../input"))

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../input'

In [147]:
import pandas as pd
import numpy as np
import tensorflow as tf
from IPython.display import clear_output

In [148]:
# Training Data
train = pd.read_csv("../input/titanic/train.csv",
                 na_values=['', '?', '-'])
# Testing Data
test = pd.read_csv("../input/titanic/test.csv",
                 na_values=['', '?', '-'])
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


###  Cleaning and filling data


In [149]:
# Training data
train = train.fillna(method='ffill')
train = train.fillna(method='bfill')
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [152]:
# Testing Data
test =test.fillna(method='bfill')
test = test.fillna(method='ffill')
test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [153]:
labels = train.pop('Survived')
labels.head()
train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,C85,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,C85,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,C123,S


In [154]:
CATEGORICAL_COLUMN = ["Name" , "Sex" , "Ticket","Cabin","Embarked"]
NUMERICAL_COLUMN = ["PassengerId" , "Pclass","Age","SibSp","Parch", "Fare"]
features_column = []

for feature_name in CATEGORICAL_COLUMN :
    vocab = x_train[feature_name].unique()
    features_column.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name,vocab))

for feature_name in NUMERICAL_COLUMN :
    features_column.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
s = pd.DataFrame(features_column)
s.head()

,key,vocabulary_list,dtype,default_value,num_oov_buckets
0,Name,"(Braund, Mr. Owen Harris, Cumings, Mrs. John B...",<dtype: 'string'>,-1,0.0
1,Sex,"(male, female)",<dtype: 'string'>,-1,0.0
2,Ticket,"(A/5 21171, PC 17599, STON/O2. 3101282, 113803...",<dtype: 'string'>,-1,0.0
3,Cabin,"(C85, C123, E46, G6, C103, D56, A6, C23 C25 C2...",<dtype: 'string'>,-1,0.0
4,Embarked,"(S, C, Q)",<dtype: 'string'>,-1,0.0


In [155]:
# Input Function
def making_data_ready(data_df , label_df , num_epochs=10, shuffle=True , batch_size=32):
    def input_function(): #inner function, this will be returned
        # Create tf.data.Dataset object with data and its label
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle :
            ds = ds.shuffle(1000) #random order of data
        ds = ds.batch(batch_size).repeat(num_epochs)#split dataset into batches of 32 and repeat process for number of epochs
        return ds # return a batch of dataset
    return input_function # return a function object for use

# here we will call the input function that was return to
train_eval = making_data_ready(train , labels) 
# test_eval = making_data_ready(x_test , y_test , num_epochs=1 , shuffle=False)

In [156]:
# Creating Model

# we are creating linear estimator by passing feature_columns we created earlier. 
linear_est = tf.estimator.LinearClassifier(feature_columns=features_column)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\india\\AppData\\Local\\Temp\\tmpaghttnn0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [194]:
# Training
linear_est.train(train_eval) #train


INFO:tensorflow:Calling model_fn.


c:\users\india\appdata\local\programs\python\python38\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\india\AppData\Local\Temp\tmpaghttnn0\model.ckpt-280
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 280...
INFO:tensorflow:Saving checkpoints for 280 into C:\Users\india\AppData\Local\Temp\tmpaghttnn0\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 280...
INFO:tensorflow:loss = 0.84694463, step = 280
INFO:tensorflow:global_step/sec: 195.84
INFO:tensorflow:loss = 0.79023826, step = 380 (0.511 sec)
INFO:tensorflow:global_step/sec: 550.934
INFO:tensorflow:loss = 0.4579243, step = 480 (0.181 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 560...
INFO:tensorflow:Saving checkpoints for 560 into C:\Users\india\AppData\Local\Temp\tmpaghttnn0\model.ckpt.
INFO:tenso

In [195]:
def predict_labels(features , batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [196]:
predict_label = lambda:predict_labels(test)
predictions = linear_est.predict(predict_label)

In [197]:
predict = pd.DataFrame(predictions)
predict

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\india\AppData\Local\Temp\tmpaghttnn0\model.ckpt-560
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


,logits,logistic,probabilities,class_ids,classes,all_class_ids,all_classes
0,[-3.587676],[0.026917905],"[0.9730821, 0.026917925]",[0],[b'0'],"[0, 1]","[b'0', b'1']"
1,[-0.62884766],[0.34777188],"[0.65222806, 0.34777185]",[0],[b'0'],"[0, 1]","[b'0', b'1']"
2,[-3.5435386],[0.028098494],"[0.9719015, 0.028098492]",[0],[b'0'],"[0, 1]","[b'0', b'1']"
3,[-3.5852346],[0.02698195],"[0.9730181, 0.02698195]",[0],[b'0'],"[0, 1]","[b'0', b'1']"
4,[-0.115699396],[0.47110736],"[0.5288926, 0.47110733]",[0],[b'0'],"[0, 1]","[b'0', b'1']"
...,...,...,...,...,...,...,...
413,[-4.1965475],[0.01482439],"[0.9851756, 0.014824369]",[0],[b'0'],"[0, 1]","[b'0', b'1']"
414,[3.456069],[0.96941155],"[0.030588385, 0.9694117]",[1],[b'1'],"[0, 1]","[b'0', b'1']"
415,[-4.207988],[0.014658242],"[0.9853418, 0.014658213]",[0],[b'0'],"[0, 1]","[b'0', b'1']"
416,[-4.1903973],[0.014914453],"[0.9850855, 0.01491446]",[0],[b'0'],"[0, 1]","[b'0', b'1']"


In [206]:
survive = []
for i in range(len(predict)):
    if predict.loc[i]['probabilities'][0] >= 0.55 :
        survive.append(0)
    else:
        survive.append(1)

gender_submission = pd.DataFrame({
    "PassengerId" : test['PassengerId'],
    "Survived" : survive
})
gender_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [ ]:
submission = pd.read_csv("../input/gender_submission.csv")
submission['Survived'] = survive
submission.to_csv('submission.csv', index=False)